# Progetto di Applicazioni Data Intensive 2018/2019


__Orazi Filippo 0000801069__

__Alesiani Matteo 0000806466__


## Descrizione del problema

Il problema da noi analizzato si pone l'obbiettivo di prevedere per conto di una società che effettua bike sharing (ossia una forma di affitto di biciclette automatizzato) il numero di biciclette che saranno affittate durante una giornata che presenta determinate caratteristiche.

Il dataset denominato "Bike Sharing Data Set" è stato scaricato dal sito https://archive.ics.uci.edu/ml e si compone di 731 istanze, 16 diversi attributi di cui 2 identificatori e 3 possibili soggetti di predizione.

In [116]:
%matplotlib  inline
import os.path
import csv
import pandas as pd

if os.path.exists("day.csv"):
    ds = pd.read_csv("day.csv", sep=",")
else:
    print("File non trovato")


In [117]:
len(ds.columns)


16

### Descrizione variabili


Il dataset scelto presenta variabili strutturate, ovvero i cui valori sono noti.

Vengono di seguito descritte:

- __instant__: indice dei record.<br>
- __dteday__: data.<br>
- __season__: stagione<br>
- __yr__:  anno (0: 2011, 1: 2012) <br> 
- __mnth__: mese (1 - 12) <br>
- __holiday__: giorno festivo (1: si, 0: no)<br>
- __weekday__: giorno della settimana (0 - 6)<br>
- __workingaday__: se il giorno è festivo o appartiene al weekend 0, altrimenti 1<br>
- __wheathersit__: condizioni meteo generali della giornata:
  * 1: soleggiato, poco nuvoloso
  * 2: nuvoloso, nebbia
  * 3: leggera neve, leggera pioggia
  * 4: neve, pioggia, fulmini <br> 

- __temp__: temperatura media giornaliera (C) normalizzata. Valori divisi per 41. <br>
- __atemp__: temperatura perepita (°C) normaizzata. Valori divisi per 50<br>
- __hum__: percentuale di umidità<br>
- __windspeed__: velocità del vento normalizzata, Valori divisi per 67. <br>
- __casual__: numero di utenti casuali<br>
- __registered__: numero di utenti registrati<br>
- __cnt__: numero di utenti totali<br>

La variabile scelta come oggeto di predizione è la variabile "cnt" in quanto a fini di ricerca di mercato è la variabile che più interessa. Vengono qundi esclusi gli attributi "casual" e "registered", di cui "cnt" è la somma, e le relative colonne.

Osserviamo come la variabile da predire sia di tipo continuo. La metodologia utilizzata, per la risoluzione del problema, attua un algoritmo di regressione. 

Concludianmo la descrizione delle variabili definendo come indice del dataframe l'attrbuto "dteday" e eliminando "instant", poichè svolge la stessa funzione.

In [118]:
ds.set_index(["dteday"], inplace=True)
dataset = ds.drop(["casual", "registered","instant"], axis=1)
Y = dataset["cnt"]

## Analisi esplorativa

Il compito dell'analisi esplorativa consiste nel identificare nel dataset le caratteristiche degli attirbuti (feature) che possono influenzare la creazione del modello, ad esempio la presenza di valori nulli comporterebbe l'esigenza di attuare contromisure volte alla loro gestione. 

Altro compito dell'analisi sta nel rappresentare l'insieme dei dati attraveso indici di correlazione tra variabili, grafici e descizioni matematiche.

Possiamo ottenere un inisieme di descrittori matematici dei vari attibuti attraverso la funzione _describe_.


In [120]:
dataset.describe()


,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,2.496580,0.500684,6.519836,0.028728,2.997264,0.683995,1.395349,0.495385,0.474354,0.627894,0.190486,4504.348837
std,1.110807,0.500342,3.451913,0.167155,2.004787,0.465233,0.544894,0.183051,0.162961,0.142429,0.077498,1937.211452
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,22.000000
25%,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.337083,0.337842,0.520000,0.134950,3152.000000
50%,3.000000,1.000000,7.000000,0.000000,3.000000,1.000000,1.000000,0.498333,0.486733,0.626667,0.180975,4548.000000
75%,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,0.655417,0.608602,0.730209,0.233214,5956.000000
max,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.861667,0.840896,0.972500,0.507463,8714.000000


La Funzione _info_ di _pandas.Dataframe_ permette di conoscere un ulteriore insieme di informazioni del dataframe

In [122]:
dataset.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 731 entries, 2011-01-01 to 2012-12-31
Data columns (total 12 columns):
season        731 non-null int64
yr            731 non-null int64
mnth          731 non-null int64
holiday       731 non-null int64
weekday       731 non-null int64
workingday    731 non-null int64
weathersit    731 non-null int64
temp          731 non-null float64
atemp         731 non-null float64
hum           731 non-null float64
windspeed     731 non-null float64
cnt           731 non-null int64
dtypes: float64(4), int64(8)
memory usage: 116.4 KB


Proseguiamo l'analisi del Dataset verificando la correlazione che l'attributo da predire ha in relazione agli altri. La correlazione di due variabili casuali X e Y, è dato dal rapporto tra la loro covarianza σXY e il prodotto delle deviazioni standard σX e σY
ρ(X,Y)= σXYσXσY

Il coefficiente ha un valore compreso tra 1 e -1, dove
valori vicini a 1 indicano correlazione diretta (Y cresce al crescere di X)
valori vicini a -1 indicano correlazione inversa (Y descresce al crescere di X)
valori vicini a 0 indicano assenza di correlazione

Le seguanti funzioni consentono di calcolare la correlazione tra due serie e il grafico relativo:

In [ ]:
import numpy as np
def getCorrelation(feature1, feature2):
    return np.mean((feature1-feature1.mean()) * (feature2-feature2.mean())) / (feature1.std() * feature2.std())


In [ ]:
import matplotlib.pyplot as plot
def plotData(x, y, XAxisName, YAxisName):
    plot.scatter(x, y)
    plot.grid()
    plot.xlabel(XAxisName); plot.ylabel(YAxisName)
    plot.show()


Mentre con la seguente si può ottenere una serie ordinata che indica la correlazione tra "cnt" e il nostro dataset indicizzato su "dtaday", un histogramma di ogni attributo, e un grafico a dispersione di ogni feature con l'obbiettivo


In [ ]:
def correlationRank(dataset, feature):
    correlation = []
    for a in dataset.columns:
        correlation.append(getCorrelation(dataset[a].astype("float"), feature))
        plotData(dataset[a].astype("float"), feature, a, "Byke Rent")
    cor = pd.Series(correlation, dataset.columns)
    cor.sort_values(ascending=False, inplace=True)
    return cor


In [ ]:
cor = correlationRank(dataset.drop(["cnt"], axis=1),dataset["cnt"])

In [ ]:
print(cor)

Dai grafici e dal calcolo della correlazione scopriamo che gli attributi weekday workingday e holiday sono attributi poco importanti per il calcolo.
Nonostante ciò essendo la correlazione un calcolo su un coefficente di primo grado questi attributi poco correlati non vengono esclusi dal calcolo in quanto questo si baserà verosimilmente su un algoritmo polinomiale.

## Preaparazione dei dati

Molti dei dati sono già stati standardizzati alla creazione del dataset, i dati che normalmente vengono presentati come categorici sono già forniti in forma numerica

I dati che necessitano di standardidazione verranno elaborati successivamente in ogni Pipeline attraverso la funzione di sklearn _StandardScaler_

essendo questo un problema di regressione calcoliamo con la norma L1 le feature più rilevanti ma prima dividiamo il dataset in trainSet e ValidationSet


In [ ]:
from sklearn.model_selection import train_test_split
Y = dataset["cnt"]
X = dataset.drop(["cnt"], axis=1)
XTrain, XVal, YTrain, YVal = train_test_split(X, Y, test_size=0.33, random_state=73)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Lasso

def elaborationWithLasso(degeePipe=1, alphaPipe=0):
    return Pipeline([("poly", PolynomialFeatures(degree=degeePipe, include_bias=False)),
                    ("scale",  StandardScaler()),
                    ("linreg", Lasso(alpha=alphaPipe, max_iter=6000, tol=0.005))])


In [ ]:
def showZerosFeatures(XTrain, YTrain):
    model = elaborationWithLasso(1, 2)
    model.fit(XTrain, YTrain)
    tmp = pd.Series(model.named_steps["linreg"].coef_, XTrain.columns)
    print(tmp)
    a = []
    for row in tmp.index:
        if(tmp[row]==0):
            a.append(row)
    print(a)


In [ ]:
showZerosFeatures(XTrain, YTrain)


Prima di decidere se mantenere o no la colonna "temp" valutiamo la bontà del modello attraverso il calcolo del coefficiente $R^2$, dell' errore relativo e dell'errore quadratico medio.

In [ ]:
def relativeError(YTrue, YPred):
    return np.mean(np.abs((YTrue - YPred) / YTrue))


In [ ]:
from sklearn.metrics import mean_squared_error
def printEvalutation(X, Y, model):
    print("Mean squared error    : {:.5}".format(mean_squared_error(model.predict(X), Y)))
    print("Relative error        : {:.5%}".format(relativeError(model.predict(X), Y)))
    print("R-squared coefficient : {:.5}".format(model.score(X, Y)))


In [ ]:
model = elaborationWithLasso(6, 8)
model.fit(XTrain, YTrain)
printEvalutation(XVal, YVal, model)
#model.named_steps["linreg"].coef_

Il risultato è buono ma sono possibili miglioramenti per cui non andremo ad escludere manualmente gli attributi che la norma L1 azzera ma piuttosto riuseremo la norma successivamente.

## Generazione modelli di learning


Generiamo adesso diversi modelli di learning utilizzando k (nested) cross fold validation applicata ad una grid search.

Definiamo innanzitutto le pipeline di ogni modello.


Il modello Ridge è un modello di regressione lineare che applica $ \Vert \theta \Vert_2^2 = \sum_{i=1}^n  \theta _i ^2$ ossia la norma l2 in modo da regolarizzare le dimensioni dei coefficienti del modello.
Questo permette di ridurre le oscillazione ed aumentare l'accuratezza del modello

In [ ]:
from sklearn.linear_model import Ridge

def elaborationWithRidge():
    return Pipeline([("poly", PolynomialFeatures(include_bias=False)),
                    ("scale",  StandardScaler()),   
                    ("linreg", Ridge())])

Il modello Elastic Net è un modello di regressione lineare che applica si la norma l1 sia la norma l2 in questo modo $ \alpha \Vert \theta \Vert_1 + (1 - \alpha) \Vert \theta \Vert_2$.
l'iperparametro $\alpha$ indica quanto il modello è "sbilanciato verso la norma L1

In [ ]:
from sklearn.linear_model import ElasticNet

def elaborationWithElasticNet():
    return Pipeline([("poly",   PolynomialFeatures(include_bias=False)),
                     ("scale",  StandardScaler()),
                     ("linreg",  ElasticNet(tol = 0.05, max_iter = 6000))])

Creiamo anche un modello di regressione lineare senza restrizioni

In [ ]:
from sklearn.linear_model import LinearRegression

def elaborationWithoutRestrain():
    return Pipeline([("poly",  PolynomialFeatures(include_bias=False)),
                    ("scale",  StandardScaler()),
                    ("linreg", LinearRegression())])

Ora definiamo il grado e gli iperparametri migliori attraverso una gridsearch con cross validation e addestriamo i modelli

In [ ]:
from sklearn.model_selection import GridSearchCV

parRidge = {
    "poly__degree": [1,6,8],
    "linreg__alpha":  [1,2,6]
}
model = elaborationWithRidge()
ridgeGridSearch = GridSearchCV(model, param_grid=parRidge)
ridgeGridSearch.fit(XTrain, YTrain)
print(ridgeGridSearch.best_params_)


In [ ]:
parLasso = {
    "poly__degree": [1,6,8],
    "linreg__alpha":  [1,5,8]
}
LassoModel = elaborationWithLasso()
lassoGridSearch = GridSearchCV(LassoModel, param_grid=parLasso)
lassoGridSearch.fit(XTrain, YTrain)
print(lassoGridSearch.best_params_)


In [ ]:
parNet = {
    "poly__degree": [1,2,6],
    "linreg__alpha": [1,2,8],
    "linreg__l1_ratio": [0.1, 0.5, 1.0]
}
NETmodel = elaborationWithElasticNet()
gs = GridSearchCV(NETmodel, param_grid=parNet)
gs.fit(XTrain, YTrain)
print(gs.best_params_)


In [ ]:
from sklearn.model_selection import GridSearchCV

parNR = {
   "poly__degree": [1,2],
}
NRmodel = elaborationWithoutRestrain()
NRGridSearch = GridSearchCV(NRmodel, param_grid=parNR)
NRGridSearch.fit(XTrain, YTrain)
print(NRGridSearch.best_params_)


## Valutazione modelli


Valutiamo ora i modelli ricavati nel punto precedente attraverso le metriche gia introdotte di $R^2$, errore relativo e errore quadratico medio. aggiungiamo alla valutazione una tabella ottenuta attraverso l'attributo *cv_results* di _GridSearchCV_ in modo da verificare quali parametri hanno portato un risultato migliore nei vari tipi di regressione. 

In [ ]:
def EvalutationTable(results):
    return pd.DataFrame(results.cv_results_).sort_values("mean_test_score", ascending=False)

In [ ]:
printEvalutation(XVal, YVal, lassoGridSearch)
EvalutationTable(lassoGridSearch)

In [ ]:
printEvalutation(XVal, YVal, ridgeGridSearch)
EvalutationTable(ridgeGridSearch)

In [ ]:
printEvalutation(XVal, YVal, NRGridSearch)
EvalutationTable(NRGridSearch)

In [ ]:
printEvalutation(XVal, YVal, gs)
EvalutationTable(gs)

definiamo come modelli migliori i seguenti: <br>
* regressione con lasso di grado 6 e con $\lambda$ = 8
* regressione con lasso di grado 6 e con $\lambda$ = 5
* regressione con Elastic net di grado 6 con $\lambda = 2$ e $\alpha = 0.5$
 
che presentano rispettivamente le seguenti metriche di giudizio


In [ ]:
LassoModel1 = Pipeline([("poly", PolynomialFeatures(degree=6, include_bias=False)),
                        ("scale",  StandardScaler()),
                        ("linreg", Lasso(alpha=8, max_iter=6000, tol=0.005))])
print("LassoModel1")
LassoModel1.fit(XTrain, YTrain)
count = 0
for a in LassoModel1.named_steps["linreg"].coef_ :
    if a != 0:
        count += 1
print(count)
printEvalutation(XVal, YVal, LassoModel1)

print("\nLassoModel2")
LassoModel2 = Pipeline([("poly", PolynomialFeatures(degree=6, include_bias=False)),
                        ("scale",  StandardScaler()),
                        ("linreg", Lasso(alpha=5, max_iter=6000, tol=0.005))])
LassoModel2.fit(XTrain, YTrain)
count = 0
for a in LassoModel2.named_steps["linreg"].coef_ :
    if a != 0:
        count += 1
print(count)

printEvalutation(XVal, YVal, LassoModel2)

print("\nENModel")
ENModel = Pipeline([("poly",   PolynomialFeatures(degree = 6, include_bias=False)),
                     ("scale",  StandardScaler()),
                     ("linreg",  ElasticNet(alpha=2, l1_ratio=0.5, tol = 0.05, max_iter = 6000))])
ENModel.fit(XTrain, YTrain)
count = 0
for a in ENModel.named_steps["linreg"].coef_ :
    if a != 0:
        count += 1
print(count)

printEvalutation(XVal, YVal, ENModel)


Dei tre modelli complessivamente molto buoni, viene scartato il modello con ElasticNet perchè nel complesso presenta risultati peggiori. 

Dei due modelli che utilizzano il Lasso viene designato come migliore quello che presenta grado 6 e con $\lambda$ = 8 perchè ottiene prestazioni migliori in tutte le metriche di giudizio.  
Inoltre compie la propria elaborazione con un numero di coefficienti minore, ne consegue una elaborazione più efficente rispetto al carico computazionale. 

Infine è importante considerare che la diminuzione del numero di coefficienti può portare al fenomeno dell'overfitting, ossia la costruzione di un modello troppo aderente al training set con un apparente aumento della qualità della previsione.
Un Modello così sviluppato potrebbe non risultare descrittivo quando applicato ad un dataset ignoto.